In [1]:
import xarray as xr
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
import metpy
import metpy.calc as mpcalc
from metpy.plots import SkewT
from metpy.units import units 
from IPython.display import HTML, display
import imageio
from tqdm import tqdm
from itertools import product
import wrf
import glob
from scipy.interpolate import RegularGridInterpolator, interp1d
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import TwoSlopeNorm

Cannot import USCOUNTIES and USSTATES without Cartopy installed.


In [2]:
#Create Path For Data Collection and Create A List of File Names
storm16_data_path = '/storage/work/bsh5393/storm16/'
storm16_files = os.listdir(storm16_data_path)

#Remove Unneccessary Files From Storm 16 File List
storm16_files.remove('README-TIMELEVELS')
storm16_files.remove('namelist.input')
storm16_files.remove('runstorm16.pbs.o44255112')
storm16_files.remove('cm1out_stats.nc')

#Read in Raw Trajectories

#Step 1: Initialization and Input Parameters

#Data at t = 2hrs
storm_data = xr.open_dataset(storm16_data_path + storm16_files[-1])

#Dataset Domain Size
ni = storm_data['ni']
nj = storm_data['nj']
nk = storm_data['nk']

#Datapoint Locations
x_data = storm_data['xh']*units.kilometer
y_data = storm_data['yh']*units.kilometer
z_data = storm_data['z']*units.kilometer

#Grid Locations (n+1)
x_grid = storm_data['xf']*units.kilometer
y_grid = storm_data['yf']*units.kilometer
z_grid = storm_data['zf']*units.kilometer

#Create New Lower/Upper Bound Variables Since We are not in the center of the domain anymore
lower_bound_x = np.abs(x_data.values - (x_data[-1].values-25)).argmin()-2
upper_bound_x = np.abs(x_data.values - x_data[-1].values).argmin()
lower_bound_y = np.abs(y_data.values - 5).argmin()
upper_bound_y = np.abs(y_data.values - 30).argmin()+2
horiz_length = upper_bound_y-lower_bound_y

#Grid Size 
size_x = size_y = upper_bound_x-lower_bound_x
size_z = len(z_data)

#Grid Spacing 
dx = x_data[1].values - x_data[0].values
dy = y_data[1].values - y_data[0].values
dz = np.zeros(len(z_data))
dz[0] = z_data[0].values
dz[1:] = z_data[1:].values-z_data[0:-1].values

#Time Intervals of Data
time = []
for x in np.arange(1,len(storm16_files),1):
    t = xr.open_dataset(storm16_data_path + storm16_files[x])['time']
    time.append(t)    
time = (np.asarray(time)*10**-9).reshape(len(time))
time = time.astype('int')
time = time - time[0]+30 #time elapsed in seconds (useful for interpolation later on)

#Time Intervals of Interpolation (We want every 10 seconds)
int_time = np.arange(int(time[0]), int(time[-1]), 10)
nanal = len(int_time) #Number of Trajectory Points 
dt = int_time[1]-int_time[0]

#Input Parameters 
asc_rate = 5 #3 m/s ascent rate of balloon


In [6]:
#Load The pseudo_data

path = '/storage/work/bsh5393/Variability Study/Sensitivity Analysis/Data/'
pseudo_data = np.load(path+'pseudodata_2_cm1grid.npz')
points_data = np.load(path+'instantaneous_data.npz')


#Data Interpolated to Model Grid
P = pseudo_data['P']
T = pseudo_data['T']
Th = pseudo_data['Th']
Td = pseudo_data['Td']
qv = pseudo_data['qv']
u = pseudo_data['u']
v = pseudo_data['v']
w = pseudo_data['w']
zvort = pseudo_data['zvort']
parc_T = pseudo_data['parc_T']
CAPE = pseudo_data['CAPE']
CIN = pseudo_data['CIN']
SRH1km = pseudo_data['SRH1km']
SRH3km = pseudo_data['SRH3km']

#Original Trajectory Points
# points = pseudo_data['interp_points']

#Ground Relative Winds
offset_u = 12.2 
offset_v = 12.5 
gr_u = u[0] + offset_u
gr_v = v[0] + offset_v
gr_wind = np.sqrt((gr_u**2)+(gr_v**2)) 

inst_data = np.load(path+'instantaneous_data.npz')
points_data = np.load(path+'interpolation_2_rawtrajectories.npz')

inst_P = inst_data['P']
inst_T = inst_data['T']
inst_Td = inst_data['Td']
inst_Th = inst_data['Th']
inst_qv = inst_data['qv']
inst_u = inst_data['u']
inst_v = inst_data['v']
inst_w = inst_data['w']
inst_zvort = inst_data['zvort']
inst_parc_T = inst_data['parc_T']
inst_CAPE = inst_data['CAPE']
inst_CIN = inst_data['CIN']
inst_SRH1km = inst_data['SRH1km']
inst_SRH3km = inst_data['SRH3km']

#Original Trajectory Points
points = points_data['interp_points']

#Ground Relative Winds
offset_u = 12.2
offset_v = 12.5 
gr_u = inst_u[0]+offset_u
gr_v = inst_v[0]+offset_v
inst_gr_wind = np.sqrt((gr_u**2)+(gr_v**2))

In [5]:
#Try to Plot Multiple (Tweak This To Pick 100 Random Soundings)

path = '/storage/work/bsh5393/Variability Study/Sensitivity Analysis/Figure Builder/'
gifname = '/storage/work/bsh5393/Variability Study/Sensitivity Analysis/GIFs/100randpseudosoundings.gif'


for x in tqdm(range(100)):
    
    k = np.random.randint(0,335**2)
    image_filename = "figure_%03d.png" % (x)
    # Plot the data using normal plotting functions, in this case using
    # log scaling in Y, as dictated by the typical meteorological plot.
    fig = plt.figure(figsize=(15,10))
    skew = SkewT(fig, rotation=45)
    skew.ax.set_ylim(1000, 100)
    skew.ax.set_xlim(-40, 60)
    skew.plot(P[:,k]*units.Pa, T[:,k]*units.kelvin, 'r')
    skew.plot(P[:,k]*units.Pa, Td[:,k]*units.kelvin, 'r')
    skew.plot_barbs(P[:,k]*units.Pa, u[:,k]*units('m/s'), v[:,k]*units('m/s'))

    # Set some better labels than the default
    skew.ax.set_title(f'100 Random PseudoSoundings (ID: {k})')
    skew.ax.set_xlabel(f'Temperature ({(T[:,k]*units.kelvin).units:~P})')
    skew.ax.set_ylabel(f'Pressure ({(P[:,k]*units.Pa).units:~P})')

    # Calculate full parcel profile and add to plot as black line
    #prof = mpcalc.parc_T(p_avg, T_avg[0], Td_avg[0]).to('degC')
    skew.plot(P[:,k]*units.Pa, parc_T[:,k]*units.kelvin, 'r--', linewidth=2)

    # Add the relevant special lines
    #skew.plot_dry_adiabats()
    #skew.plot_moist_adiabats()
    #skew.plot_mixing_lines()

    #Relative Storm Motion 
    offset_u = 12.2*units.meter_per_second
    offset_v = 12.5*units.meter_per_second
    #Add Hodograph
    ax_hodograph = fig.add_axes([.84, 0.7, 0.18, 0.18])
    h = metpy.plots.Hodograph(ax_hodograph, component_range=40.)
    h.add_grid(increment=5)
    h.plot(u[:, k] * units('m/s')+offset_u, v[:, k] * units('m/s')+offset_v, color='tab:red')
    
    #Save the figure
    fig.savefig(path + image_filename)
    
    #clear the figure
    plt.close()

#These variables control the time step (interval) and the length of the pause at the end of the animation (end_interval)
interval = 0.25
end_interval = 1.0

#Iterating through all of the files in path, and then sorting the filenames
imagefiles = os.listdir(path)
imagefiles.remove('.ipynb_checkpoints')


image_filenames = []

for image_filename in imagefiles[:x]:
    image_filenames.append(image_filename)
image_filenames.sort()
    
#Opening each image file and saving it to a list
images = []
for image_filename in image_filenames[:x]:
    images.append(imageio.imread(path + image_filename))
      
#Generating an array containing the duration to display each frame
durations = np.ones(len(images))*interval
durations[-1] = end_interval

#Generating the animation itself
imageio.mimsave(gifname, images, duration = durations.tolist())

100%|██████████| 100/100 [00:26<00:00,  3.76it/s]


In [8]:
#Create A GIF That Randomly Selects 100 Pseudo & Instantanous 

path = '/storage/work/bsh5393/Variability Study/Sensitivity Analysis/Figure Builder/'
gifname = '/storage/work/bsh5393/Variability Study/Sensitivity Analysis/GIFs/100randsoundingscompared.gif'

for x in tqdm(range(100)):
    #Choose Random Sounding ID 
    k = np.random.randint(0,P.shape[1])
    image_filename = "figure_%03d.png" % (x)
    
    #figure initialization
    fig = plt.figure(figsize = (15,10))
    skew = SkewT(fig, rotation = 45)
    
    
    #PSEUDO
    skew.plot(P[:,k]*units.Pa, T[:,k]*units.degK, 'g')
    skew.plot(P[:,k]*units.Pa, Td[:,k]*units.degK, 'g')
    skew.plot(P[:,k]*units.Pa, parc_T[:,k]*units.kelvin, 'g.', linewidth=2)
    #skew.plot_barbs(P[:,k]*units.Pa, u[:,k]*units('m/s'), v[:,k]*units('m/s'))
    skew.ax.set_ylim(1000,100)
    skew.ax.set_xlim(-40,60)
    
    #cosmetics
    skew.ax.set_title(f'100 Randomly Selected Pseudo (g) vs. Instantaneous (r) Soundings (ID: {k})')
    skew.ax.set_xlabel(f'Temperature ({(T[:,k]*units.kelvin).units:~P})')
    skew.ax.set_ylabel(f'Pressure ({(P[:,k]*units.Pa).units:~P})')
    
    #INSTANTANEOUS
    skew.plot(inst_P[:,k]*units.Pa, inst_T[:,k]*units.degK, 'r')
    skew.plot(inst_P[:,k]*units.Pa, inst_Td[:,k]*units.degK, 'r')
    skew.plot(inst_P[:,k]*units.Pa, inst_parc_T[:,k]*units.kelvin, 'r--', linewidth=2)
    #skew.plot_barbs(inst_P[:,k]*units.Pa, inst_u[:,k]*units('m/s'), inst_v[:,k]*units('m/s'))
    skew.ax.set_ylim(1000,100)
    skew.ax.set_xlim(-40,60)
    
    #cosmetics
    skew.ax.set_xlabel(f'Temperature ({(inst_T[:,k]*units.kelvin).units:~P})')
    skew.ax.set_ylabel(f'Pressure ({(inst_P[:,k]*units.Pa).units:~P})')
    
    #Plot Hodographs 
    ax_hodograph = fig.add_axes([.84, 0.7, 0.18, 0.18])
    h = metpy.plots.Hodograph(ax_hodograph, component_range = 40.)
    h.add_grid(increment = 5)
    h.plot(inst_u[:,k]+offset_u, inst_v[:,k]+offset_v, color='tab:green')
    h.plot(u[:,k]+offset_u, v[:,k]+offset_v, color='tab:red')
    
    #Save Figure
    fig.savefig(path+image_filename)
    
    #Clear Figure 
    plt.close()
    
    #Combine Above Images into GIF
#These variables control the time step (interval) and the length of the pause at the end of the animation (end_interval)
interval = 0.35
end_interval = 1.0

#Iterating through all of the files in path, and then sorting the filenames
imagefiles = os.listdir(path)
imagefiles.remove('.ipynb_checkpoints')


image_filenames = []

for image_filename in imagefiles[:x]:
    image_filenames.append(image_filename)
image_filenames.sort()
    
#Opening each image file and saving it to a list
images = []
for image_filename in image_filenames[:x]:
    images.append(imageio.imread(path + image_filename))
      
#Generating an array containing the duration to display each frame
durations = np.ones(len(images))*interval
durations[-1] = end_interval

#Generating the animation itself
imageio.mimsave(gifname, images, duration = durations.tolist())

100%|██████████| 100/100 [00:28<00:00,  3.54it/s]
